In [1]:
from utils import Dataset, Graphsize
from tqdm import tqdm
from tqdm.notebook import tqdm as tqdm_notebook
from time import time
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [264]:
webkb = Dataset('/home/mangaravite/Documents/datasets/topics/webkb/')
_20ng = Dataset('/home/mangaravite/Documents/datasets/topics/20ng/')
acm   = Dataset('/home/mangaravite/Documents/datasets/topics/acm/')
reut  = Dataset('/home/mangaravite/Documents/datasets/topics/reut/')

dataset = reut

In [265]:
fold = next(dataset.get_fold_instances(5))
fold._fields

('X_train', 'y_train', 'X_test', 'y_test', 'X_val', 'y_val')

In [266]:
%%time
max_feats = 25000
graph_builder = Graphsize(w=5, verbose=True, max_feat=max_feats, feature_type='full_weight_prob')
Gs_train = graph_builder.fit_transform(fold.X_train)
Gs_val   = graph_builder.transform(fold.X_val)

100%|██████████| 2702/2702 [00:04<00:00, 573.61it/s]

CPU times: user 2min 19s, sys: 1.22 s, total: 2min 21s
Wall time: 2min 20s


In [267]:
Gs_test  = graph_builder.transform(fold.X_test)

100%|██████████| 2702/2702 [00:04<00:00, 564.19it/s]


In [268]:
import torch
import dgl
import dgl.function as fn
import torch.nn as nn
import torch.nn.functional as F
from dgl.nn.pytorch.conv import GraphConv, GATConv
from dgl.nn.pytorch.glob import GlobalAttentionPooling

import torch.optim as optim
from torch.utils.data import DataLoader

In [269]:
class SimpleClassifierGCN(nn.Module):
    def __init__(self, in_dim, hidden_dim, n_classes, drop=.5):
        super(SimpleClassifierGCN, self).__init__()

        self.layers = nn.ModuleList([
            #GraphConv(in_dim, hidden_dim, activation=F.leaky_relu).to(torch.device('cuda:0')),
            #GraphConv(hidden_dim, hidden_dim, activation=F.leaky_relu).to(torch.device('cuda:0'))
            GraphConv(in_dim, hidden_dim, activation=F.relu).to(torch.device('cuda:0')),
            GraphConv(hidden_dim, hidden_dim, activation=F.relu).to(torch.device('cuda:0'))
        ])
        
        self.lin = nn.Linear(hidden_dim, 1).to(torch.device('cuda:0'))
        
        self.norm = nn.BatchNorm1d( hidden_dim )
        self.drop = nn.Dropout(drop)
        
        self.classify = nn.Linear(hidden_dim, n_classes).to(torch.device('cuda:0'))

    def forward(self, G):
        h = G.ndata['f']
        for conv in self.layers:
            h = conv(G, h)
            #h = h.view(h.shape[0], -1)
        G.ndata['h'] = h
        w = self.lin( h )
        G.ndata['w'] = w
        hg = dgl.mean_nodes(G, 'h', weight='w')
        
        pred = self.classify( hg )
        #pred = torch.softmax(pred, 1)
        return pred
    def transform(self, G):
        h = G.ndata['f']
        for conv in self.layers:
            h = conv(G, h)
            #h = h.view(h.shape[0], -1)
        hg = self.norm( h )
        hg = self.drop( hg )
        hg = self.pooling(G, hg)
        return hg
class ClassifierGCN(nn.Module):
    def __init__(self, in_dim, hidden_dim, n_classes, drop=.5):
        super(ClassifierGCN, self).__init__()

        self.encoder = nn.Linear(in_dim, hidden_dim).to(torch.device('cuda:0'))
        
        self.layers = nn.ModuleList([
            GraphConv(hidden_dim, hidden_dim, activation=F.relu).to(torch.device('cuda:0')),
            GraphConv(hidden_dim, hidden_dim, activation=F.relu).to(torch.device('cuda:0'))
        ])
        
        self.lin = nn.Linear(2*hidden_dim, 1).to(torch.device('cuda:0'))
        self.pooling = GlobalAttentionPooling( self.lin ).to(torch.device('cuda:0'))
        
        self.norm = nn.BatchNorm1d( 2*hidden_dim )
        self.drop = nn.Dropout(drop)
        
        self.classify = nn.Linear(2*hidden_dim, n_classes).to(torch.device('cuda:0'))

    def forward(self, G):
        h = G.ndata['f']
        he = self.encoder(h)
        h = he
        for conv in self.layers:
            h = conv(G, h)
        
        # CONCAT he E hg
        hg = torch.cat((h,he), 1)
        hg = self.norm( hg )
        hg = self.drop( hg )
        hg = self.pooling(G, hg)
        
        pred = self.classify( hg )
        return pred
    
class ClassifierGAT(nn.Module):
    def __init__(self, in_dim, hidden_dim, n_classes, n_heads=16, drop=.5):
        super(ClassifierGAT, self).__init__()

        self.layers = nn.ModuleList([
            GATConv(in_dim, hidden_dim, num_heads=n_heads, activation=F.leaky_relu,
                    feat_drop=drop, attn_drop=drop).to(torch.device('cuda:0')),
            GATConv(n_heads*hidden_dim, hidden_dim, num_heads=n_heads, activation=F.leaky_relu,
                    feat_drop=drop, attn_drop=drop).to(torch.device('cuda:0'))
        ])
        
        self.lin = nn.Linear(n_heads*hidden_dim, 1).to(torch.device('cuda:0'))
        self.pooling = GlobalAttentionPooling( self.lin ).to(torch.device('cuda:0'))
        
        self.norm = nn.BatchNorm1d( n_heads*hidden_dim )
        self.drop = nn.Dropout(drop)
        
        self.classify = nn.Linear(n_heads*hidden_dim, n_classes).to(torch.device('cuda:0'))

    def forward(self, G):
        h = G.ndata['f']
        for conv in self.layers:
            h = conv(G, h)
            h = h.view(h.shape[0], -1)
        hg = self.norm( h )
        hg = self.drop( hg )
        hg = self.pooling(G, hg)
        pred = self.classify( hg )
        return pred

In [270]:
a = torch.tensor( [[1,2,3,4,5],[5,4,3,2,1]] )
b = torch.tensor( [[-1,-2,-3,-4,-5],[-5,-4,-3,-2,-1]] )

In [271]:
def collate(samples):
    Gs_Fs, labels = map(list, zip(*samples))
    graphs = []
    for g, f in Gs_Fs:
        g_dgl = dgl.DGLGraph()
        g_dgl.from_networkx(g)
        g_dgl.ndata['f'] = torch.FloatTensor(f.A).to(torch.device('cuda:0'))
        g_dgl.to(torch.device('cuda:0'))
        graphs.append(g_dgl)
    batched_graph = dgl.batch(graphs)
    batched_graph.to(torch.device('cuda:0'))
    labels = torch.tensor(labels).to(torch.device('cuda:0'))
    return batched_graph, labels

In [272]:
class FocalLoss(nn.Module):
    # https://github.com/mbsariyildiz/focal-loss.pytorch
    def __init__(self, gamma=0, alpha=None, size_average=True):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha
        if isinstance(alpha, (float, int)): self.alpha = torch.Tensor([alpha, 1 - alpha])
        if isinstance(alpha, list): self.alpha = torch.Tensor(alpha)
        self.size_average = size_average

    def forward(self, input, target):
        if input.dim()>2:
            input = input.view(input.size(0), input.size(1), -1)  # N,C,H,W => N,C,H*W
            input = input.transpose(1, 2)                         # N,C,H*W => N,H*W,C
            input = input.contiguous().view(-1, input.size(2))    # N,H*W,C => N*H*W,C
        target = target.view(-1, 1)

        logpt = F.log_softmax(input, dim=1)
        logpt = logpt.gather(1,target)
        logpt = logpt.view(-1)
        pt = logpt.exp()

        if self.alpha is not None:
            if self.alpha.type() != input.data.type():
                self.alpha = self.alpha.type_as(input.data)
            at = self.alpha.gather(0, target.data.view(-1))
            logpt = logpt * at

        loss = -1 * (1 - pt)**self.gamma * logpt
        if self.size_average: return loss.mean()
        else: return loss.sum()

In [277]:
PATH = 'best_param_simple_reut.pth'
n_epochs = 100
patience = 25
hidden_dim = 300
train_batch_size = 16
test_val_batch_size = 256

#model = SimpleClassifierGCN(len(graph_builder.vocab), hidden_dim, dataset.nclass, drop=.5).to(torch.device('cuda:0'))
#model = ClassifierGAT(len(graph_builder.vocab), hidden_dim, dataset.nclass, n_heads=2, drop=.5).to(torch.device('cuda:0'))
model = ClassifierGCN(len(graph_builder.vocab), hidden_dim, dataset.nclass, drop=.5).to(torch.device('cuda:0'))

In [278]:
#loss_func = FocalLoss().to(torch.device('cuda:0'))
loss_func = nn.CrossEntropyLoss().to(torch.device('cuda:0'))

optimizer = optim.Adam( model.parameters(), lr=1e-3, weight_decay=1e-3)
#optimizer = optim.AdamW( model.parameters(), lr=1e-2, weight_decay=1e-3)

#optimizer = optim.RMSprop( model.parameters(), lr=1e-2, weight_decay=1e-4)
#optimizer = optim.RMSprop( model.parameters(), lr=0.0001 )

model.train()
torch.cuda.synchronize()
epoch_losses = []

In [279]:
data_loader_val  = DataLoader(list(zip(Gs_val,  fold.y_val )), batch_size=test_val_batch_size,
                              shuffle=True, collate_fn=collate)

In [280]:
best_score = None
n_iters = 0

for epoch in range(n_epochs):
    data_loader = DataLoader(list(zip(Gs_train, fold.y_train)), batch_size=train_batch_size,
                             shuffle=True, collate_fn=collate)
    epoch_loss = 0
    with tqdm(total=len(data_loader.dataset), smoothing=0.) as pbar:
        t0 = time()
        total = 0
        correct = 0
        model.train()
        for i, (bg, label) in enumerate(data_loader):
            outputs = model(bg)
            probs_Y = torch.softmax(outputs, 1)
            sampled_Y = torch.argmax(probs_Y, 1).reshape(-1)
            
            # Train eval phase
            total += label.size(0)
            correct += (sampled_Y == label).sum().item()
            
            # NN backprop phase
            loss = loss_func(outputs, label)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            epoch_loss += loss.detach().item()
            
            del loss, outputs, bg, probs_Y, sampled_Y
            pbar.update( len(label) )
            pbar.set_description_str('iter {}, train acc {:.3f} train loss {:.2f}'.format(epoch, (correct/total), epoch_loss / (epoch + 1)))
        
        score_train = correct/total
    with tqdm(total=len(data_loader_val.dataset), smoothing=0.) as pbar:
        model.eval()
        total = 0
        correct = 0
        epoch_loss = 0.
        for bg, label in data_loader_val:
            with torch.no_grad():
                outputs = model(bg)
            
            probs_Y = torch.softmax(outputs, 1)
            sampled_Y = torch.argmax(probs_Y, 1).reshape(-1)

            # Validation eval phase
            total += label.size(0)
            correct += (sampled_Y == label).sum().item()
            
            #break
            
            del probs_Y, outputs, bg, sampled_Y
            pbar.update( label.size(0) )
            score_val = correct/total

            pbar.set_description_str('iter {}, val   acc {:.3f} ( over: {:.3} )'.format(epoch, score_val, score_val/score_train))
            
        #break
        pbar.set_description_str('iter {}, val  acc {:.3f} ( over: {:.3}/{} )'.format(epoch, score_val, score_val/score_train, n_iters))
        score = correct/total
        if best_score is None or score > best_score:
            torch.save(model, PATH)
            best_score = score
            n_iters = 0
        else:
            n_iters += 1
            if n_iters >= patience:
                print()
                print('BEST val acc {:.3f}'.format(best_score), end='\r')
                break
        pbar.set_description_str('iter {}, val   acc {:.3f} ( over: {:.3}/{} )'.format(epoch, score_val, score_val/score_train, n_iters))
        epoch_loss /= (epoch + 1)
        epoch_losses.append(epoch_loss)

iter 36, val  acc 0.682 ( over: 0.851/24 ): 100%|██████████| 2702/2702 [00:11<00:00, 231.95it/s]

In [ ]:
# leaky_relu

#ReLU
#BEST val acc 0.702 reut  w=5 | full_weight_prob
#BEST val acc 0.625 acm   w=5 | full_weight_prob (max_feat=25k)

#BEST val acc 0.732 webkb w=5 | full_weight
#BEST val acc 0.700 reut  w=5 | full_weight
#BEST val acc 0.754 20ng  w=5 | full_weight

In [ ]:
#BEST val acc 0.623: 
PATH = 'best_param_simple_acm.pth'
n_epochs = 100
patience = 10
hidden_dim = 300
train_batch_size = 16
test_val_batch_size = 256
loss_func = nn.CrossEntropyLoss().to(torch.device('cuda:0'))
optimizer = optim.Adam( model.parameters(), lr=1e-2, weight_decay=1e-3)

In [61]:
model = torch.load(PATH)
with torch.no_grad():
    total = 0
    correct = 0
    epoch_loss = 0.
    model.eval()
    for bg, label in data_loader_val:
        outputs = model(bg)
        probs_Y = torch.softmax(outputs, 1)
        sampled_Y = torch.argmax(probs_Y, 1).reshape(-1)
        #print(probs_Y.shape, sampled_Y.shape, label.shape)

        qtd_docs = label.size(0)
        qtd_correct = (sampled_Y == label).sum().item()
        total += qtd_docs
        correct += qtd_correct

        del probs_Y, outputs, bg, sampled_Y
        pbar.update( label.size(0) )
        score_val = correct/total

        print('val acc {:.3f} val loss {:.3} ( {}/{}. over: {:.3} )'.format(score_val, epoch_loss, correct, total, score_val/score_train), end='\r')
    #break

In [ ]:
print( len(Gs_train), '+', len(Gs_val), end=' = ' )
Gs_train_val = Gs_train + Gs_val
print( len(Gs_train_val) )

print( len(fold.y_train), '+', len(fold.y_val), end=' = ' )
y_train_val = fold.y_train + fold.y_val
print( len(y_train_val) )

In [ ]:
data_loader_train_val_t  = DataLoader(list(zip(Gs_train_val, y_train_val)), batch_size=test_val_batch_size,
                              shuffle=False, collate_fn=collate)
X_train_val_all = []
model.eval()
with torch.no_grad():
    with tqdm_notebook(total=len(data_loader_train_val_t.dataset), smoothing=0.) as pbar:
        for G, label in data_loader_train_val_t:
            X_train_val_t = model.transform( G ).cpu().numpy()
            X_train_val_all.append( X_train_val_t )
            pbar.update( len(label) )

In [ ]:
X_train_val_all2 = np.concatenate( X_train_val_all )
X_train_val_all2

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV

param = {'loss': 'squared_hinge', 'C': 1, 'verbose': 0,
         'intercept_scaling': 1, 'fit_intercept': True,
         'max_iter': 1000, 'penalty': 'l2', 'multi_class': 'ovr',
         'random_state': None, 'dual': False,'tol': 0.001,
         'class_weight': None}
estimator = LinearSVC(**param)
tunning = [{'C': 2.0 ** np.arange(-5, 9, 2)}]

gs = GridSearchCV(estimator, tunning,
                n_jobs=64, refit=False,
                cv=5, iid=True,
                verbose=2, scoring='f1_micro')

gs.fit( X_train_val_all2, y_train_val )

In [ ]:
Gs_test  = graph_builder.transform(fold.X_test)
data_loader_test = DataLoader(list(zip(Gs_test, fold.y_test)), batch_size=test_val_batch_size,
                              shuffle=False, collate_fn=collate)
X_test_all = []
model.eval()
with torch.no_grad():
    with tqdm_notebook(total=len(data_loader_test.dataset), smoothing=0.) as pbar:
        for G, label in data_loader_test:
            X_test_t = model.transform( G ).cpu().numpy()
            X_test_all.append( X_test_t )
            pbar.update( len(label) )
X_test_all = np.concatenate( X_test_all )

In [ ]:
lsvm = LinearSVC( **gs.best_params_ )
lsvm.fit( X_train_val_all2, y_train_val )

y_pred = lsvm.predict( X_test_all )

In [ ]:
sum(y_pred == fold.y_test)/len(y_pred)

In [ ]:
model = torch.load(PATH)
with torch.no_grad():
    total = 0
    correct = 0
    epoch_loss = 0.
    model.eval()
    for bg, label in data_loader_test:
        outputs = model(bg)
        probs_Y = torch.softmax(outputs, 1)
        sampled_Y = torch.argmax(probs_Y, 1).reshape(-1)
        #print(probs_Y.shape, sampled_Y.shape, label.shape)

        qtd_docs = label.size(0)
        qtd_correct = (sampled_Y == label).sum().item()
        total += qtd_docs
        correct += qtd_correct

        del probs_Y, outputs, bg, sampled_Y
        pbar.update( label.size(0) )
        score_val = correct/total

        print('val acc {:.3f} val loss {:.3} ( {}/{}. over: {:.3} )'.format(score_val, epoch_loss, correct, total, score_val/score_train), end='\r')
    #break